## Train ReLayNet
RunFile of OCT segmentation

In [10]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.data_utils import get_imdb_data

#torch.set_default_tensor_type('torch.FloatTensor')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
train_data, test_data = get_imdb_data()
print("Train size: %i" % len(train_data))
print("Test size: %i" % len(test_data))

Train size: 7
Test size: 4


In [15]:
from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.solver import Solver

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

param ={
        'num_channels':1,
        'num_filters':64,
        'kernel_h':3,
        'kernel_w':7,
        'kernel_c': 1,
        'stride_conv':1,
        'pool':2,
        'stride_pool':2,
        'num_class':9
    }

exp_dir_name = 'Exp01'

relaynet_model = ReLayNet(param)
solver = Solver(optim_args={"lr": 1e-2})
solver.train(relaynet_model, train_loader, val_loader, log_nth=1, num_epochs=20, exp_dir_name=exp_dir_name)


START TRAIN.
net.forward input.shape: torch.Size([4, 1, 456, 1000])
-----------------------------------
e1: torch.Size([4, 64, 228, 500]), out1: torch.Size([4, 64, 456, 1000]), ind1: torch.Size([4, 64, 228, 500])
e2: torch.Size([4, 64, 114, 250]), out2: torch.Size([4, 64, 228, 500]), ind2: torch.Size([4, 64, 114, 250])
e3: torch.Size([4, 64, 57, 125]), out3: torch.Size([4, 64, 114, 250]), ind3: torch.Size([4, 64, 57, 125])
-----------------------------------
bn: torch.Size([4, 64, 57, 125])
-----------------------------------
d3: torch.Size([4, 64, 114, 250])
d2: torch.Size([4, 64, 228, 500])
d1: torch.Size([4, 64, 456, 1000])
-----------------------------------
 * CombinedLoss forward: target unique values: tensor([ nan,  nan,  nan,  ..., 154., 155., 156.])
target shape: torch.Size([4, 456, 1000])


AssertionError: Torch not compiled with CUDA enabled

## Save the Model

When you are satisfied with your training, you can save the model.

In [ ]:
relaynet_model.save("models/relaynet_model.model")

# Deploy Model on Test Data

In [16]:
SEG_LABELS_LIST = [
    {"id": -1, "name": "void", "rgb_values": [0, 0, 0]},
    {"id": 0, "name": "Region above the retina (RaR)", "rgb_values": [128, 0, 0]},
    {"id": 1, "name": "ILM: Inner limiting membrane", "rgb_values": [0, 128, 0]},
    {"id": 2, "name": "NFL-IPL: Nerve fiber ending to Inner plexiform layer", "rgb_values": [128, 128, 0]},
    {"id": 3, "name": "INL: Inner Nuclear layer", "rgb_values": [0, 0, 128]},
    {"id": 4, "name": "OPL: Outer plexiform layer", "rgb_values": [128, 0, 128]},
    {"id": 5, "name": "ONL-ISM: Outer Nuclear layer to Inner segment myeloid", "rgb_values": [0, 128, 128]},
    {"id": 6, "name": "ISE: Inner segment ellipsoid", "rgb_values": [128, 128, 128]},
    {"id": 7, "name": "OS-RPE: Outer segment to Retinal pigment epithelium", "rgb_values": [64, 0, 0]},
    {"id": 8, "name": "Region below RPE (RbR)", "rgb_values": [192, 0, 0]}];
    #{"id": 9, "name": "Fluid region", "rgb_values": [64, 128, 0]}];
    
def label_img_to_rgb(label_img):
    label_img = np.squeeze(label_img)
    labels = np.unique(label_img)
    label_infos = [l for l in SEG_LABELS_LIST if l['id'] in labels]

    label_img_rgb = np.array([label_img,
                              label_img,
                              label_img]).transpose(1,2,0)
    for l in label_infos:
        mask = label_img == l['id']
        label_img_rgb[mask] = l['rgb_values']

    return label_img_rgb.astype(np.uint8)

In [17]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

relaynet_model =  torch.load('models/Exp01/relaynet_epoch20.model')
out = relaynet_model(Variable(torch.Tensor(test_data.X[0:1]).cuda(),volatile=True))
out = F.softmax(out,dim=1)
max_val, idx = torch.max(out,1)
idx = idx.data.cpu().numpy()
idx = label_img_to_rgb(idx)
plt.imshow(idx)
plt.show()

img_test = test_data.X[0:1]
img_test = np.squeeze(img_test)
plt.imshow(img_test)
plt.show()

ModuleNotFoundError: No module named 'networks'